## Analizador Lexico
Un analizador lexico transforma una entrada en tokens

In [5]:
entrada = "Si si es de dia entonces mañana no es de noche"

In [7]:
from enum import Enum

class TipoToken(Enum):
    O = 1
    Y = 2
    S = 3
    N = 4
    E = 5
    P = 6

In [12]:
def analizador_lexico (entrada : str) -> list:
    #separar cadena en palabras
    tokens = entrada.lower().split()
    
    #etiquetar cada palabra con su respectivo tipo de token
    lexemas = []
    for token in tokens:
        if token == 'o':
            lexemas.append({"valor" : token , "tipo": TipoToken.O })
        elif token == 'y':
            lexemas.append({"valor" : token , "tipo": TipoToken.Y})
        elif token == 'si':
            lexemas.append({"valor" : token , "tipo": TipoToken.S})
        elif token == 'entonces':
            lexemas.append({"valor" : token , "tipo": TipoToken.E})
        elif token == 'no':
            lexemas.append({"valor" : token , "tipo": TipoToken.N})
        else :
            lexemas.append({"valor" :token , "tipo": TipoToken.P })
        
    return lexemas

result = analizador_lexico (entrada)
print(result)

[{'valor': 'si', 'tipo': <TipoToken.S: 3>}, {'valor': 'si', 'tipo': <TipoToken.S: 3>}, {'valor': 'es', 'tipo': <TipoToken.P: 6>}, {'valor': 'de', 'tipo': <TipoToken.P: 6>}, {'valor': 'dia', 'tipo': <TipoToken.P: 6>}, {'valor': 'entonces', 'tipo': <TipoToken.E: 5>}, {'valor': 'mañana', 'tipo': <TipoToken.P: 6>}, {'valor': 'no', 'tipo': <TipoToken.N: 4>}, {'valor': 'es', 'tipo': <TipoToken.P: 6>}, {'valor': 'de', 'tipo': <TipoToken.P: 6>}, {'valor': 'noche', 'tipo': <TipoToken.P: 6>}]


## Analizador Sintactico

In [9]:
class EstadosAnalizador(Enum):
    INICIO = 1
    PRECEDENTE = 2
    ERROR1 = 3
    NEGACION = 4
    CONSECUENTE = 5
    ATOMO = 6
    DISYUNCION = 7
    CONJUNCION = 8
    ERROR2 = 9
    FIN = 10

In [16]:
def analizador_sintactico(lexemas : list):
EstadoActual = EstadosAnalizador.INICIO

for lexema in lexemas:
    lexema == 1["tipo"]
    if EstadoActual == EstadosAnalizador.INICIO:
        if lexema["tipo"] == TipoToken.S:
            EstadoActual = EstadosAnalizador.PRECEDENTE
        else:
            EstadoActual = EstadosAnalizador.ERROR1
    elif EstadoActual == EstadosAnalizador.PRECEDENTE:
        if lexema == "no":
            EstadoActual = EstadosAnalizador.NEGACION
        elif lexema == "es":
            EstadoActual = EstadosAnalizador.CONSECUENTE
        else:
            EstadoActual = EstadosAnalizador.ERROR2
    elif EstadoActual == EstadosAnalizador.NEGACION:
        if lexema == "de":
            EstadoActual = EstadosAnalizador.ATOMO
        else:
            EstadoActual = EstadosAnalizador.ERROR2
    elif EstadoActual == EstadosAnalizador.CONSECUENTE:
        if lexema == "de":
            EstadoActual = EstadosAnalizador.ATOMO
        else:
            EstadoActual = EstadosAnalizador.ERROR2
    elif EstadoActual == EstadosAnalizador.ATOMO:
        if lexema == "entonces":
            EstadoActual = EstadosAnalizador.FIN
        elif lexema == "y":
            EstadoActual = EstadosAnalizador.CONJUNCION
        elif lexema == "o":
            EstadoActual = EstadosAnalizador.DISYUNCION
        else:
            EstadoActual = EstadosAnalizador.ERROR2
    elif EstadoActual == EstadosAnalizador.CONJUNCION:
        if lexema == "de":
            EstadoActual = EstadosAnalizador.ATOMO
        else:
            EstadoActual = EstadosAnalizador.ERROR2
    elif EstadoActual == EstadosAnalizador.DISYUNCION:
        if lexema == "de":
            EstadoActual = EstadosAnalizador.ATOMO
        else:
            EstadoActual = EstadosAnalizador.ERROR2
    elif EstadoActual == EstadosAnalizador.ERROR1:
        break
    elif EstadoActual == EstadosAnalizador.ERROR2:
        break
    elif EstadoActual == EstadosAnalizador.FIN:
        break
return EstadoActual

IndentationError: expected an indented block after function definition on line 1 (1613653408.py, line 2)

In [14]:
print(analizador_sintactico(analizador_lexico("cadena de ejemplo")))
frases =[
    "si es de dia o es de noche entonces existo"

]

for frase in frases:
    print(frase)
    

NameError: name 'analizador_sintactico' is not defined

#Analisador semantico